In [1]:
from os import listdir
from sys import argv

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, precision_recall_curve

from sklearn.decomposition import PCA

from skimage import feature
from skimage.feature import greycomatrix
from skimage.feature import greycoprops
from scipy.stats import entropy
from skimage.feature import local_binary_pattern

from sklearn.svm import SVC
from skimage import measure
from skimage.io import imread

import xgboost as xgb

import pandas as pd

from sklearn.preprocessing import minmax_scale as mms
from skimage.util.shape import view_as_blocks
import scipy.stats as stats

from multiprocessing import Pool, Process, Array
import multiprocessing as mp



# Inicialização de alguma variáveis

In [38]:
"""
This script should run separatly for each image base

Base 1 : normal soil [1-18] and compacted soil [19-22];
Base 2 (new base): not calibrated images
"""

base = 2 
if base == 1:
    paths = ["imgs_orig", "gts_orig"]
elif base == 2:
    paths = ["imgs", "gts"]
else:
    raise ValueError

path_imgs = sorted([ paths[0] + '/' + i for i in listdir(paths[0]) ])
path_gts = sorted([ paths[1] + '/' + i for i in listdir(paths[1]) ])

n_images = len(path_imgs)

names_VI = ["ExG", "ExGR", "CIVE", "VEG", "WI", "NGRDI"]

In [39]:
n_features = 4 * 6 * 1 # 4 posiçoes, 6 extratores, 1 VI
n_levels = 64 # 4 * 4 * 4 (discretiza em 4 cada canal de cor)

# 1) Declaração de funções (e alguns testes)

In [40]:
mode = lambda a : max(map(lambda val: (a.count(val), val), set(a)))[1]

In [41]:
normalize = lambda img, a, b : cv2.normalize(img, img, a, b, cv2.NORM_MINMAX, dtype = 1)

## 1.1) Redução de ruído

In [42]:
def neighbMax(image):
	O = np.zeros(image.shape, dtype = float)
	ns = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    

	for lin in range(512):
		for col in range(512):
			_list = []
			for n in ns:
				if lin + n[0] >= 0 and lin + n[0] < 512 and col + n[1] >= 0 and col + n[1] < 512:
					_list.append( image[ lin + n[0] ] [col + n[1] ] )
			O[lin][col] = np.average(sorted(_list, reverse = True)[:3]) # Pega os 3 maiores

	return O

In [43]:
def neighbMin(image):
	O = np.zeros(image.shape, dtype = float)
	ns = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    

	for lin in range(512):
		for col in range(512):
			_list = []
			for n in ns:
				if lin + n[0] >= 0 and lin + n[0] < 512 and col + n[1] >= 0 and col + n[1] < 512:
					_list.append( image[ lin + n[0] ] [col + n[1] ] )
			O[lin][col] = np.average(sorted(_list, reverse = False)[:3]) # Pega os 3 menores

	return O

## 1.2) BIC

In [44]:
def BIC(image, levels):
    """
    beiradas da imagem devem ser consideradas o que?
    """
    interior = np.zeros(levels, dtype = int)
    borda = np.zeros(levels, dtype = int)
    ns = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    
    lim_lin = image.shape[0]
    lim_col = image.shape[1]

    for lin in range(lim_lin):
        for col in range(lim_col): # laço do elementos da matriz
            curr = image[lin, col]
            interior[curr] += 1
            for n in ns: # laço dos vizinhos
                if lin + n[0] >= 0 and lin + n[0] < lim_lin and col + n[1] >= 0 and col + n[1] < lim_col: # teste de estar dentro da matriz
                    if image[lin + n[0], col + n[1]] != curr:
                        borda[curr] += 1
                        interior[curr] -=1
                        break
                    

    return interior, borda

## 1.3) Haralick

In [45]:
def entropy_func(P):
    return [entropy(P[:, :, 0, i].ravel()) for i in range(P.shape[3])]

In [46]:
def maxProb(P):
    return [np.max(P[:, :, 0, i]) for i in range(P.shape[3])]

In [47]:
def get_features(P):
    """
    Cada medida retorna 4 valores (1 para cada angulo).
    O valor 0 se refere ao fato de escolher sempre a mesma distância.
    """
    contrast = greycoprops(P, prop = "contrast")[0 , :]
    correlation = greycoprops(P, prop = "correlation")[0 , :]
    energy = greycoprops(P, prop = "ASM")[0 , :]
    homogeneity = greycoprops(P, prop = "homogeneity")[0 , :]
    _maxProb = maxProb(P)
    _entropy = entropy_func(P)
    
    #print(contrast, correlation, energy, homogeneity, _maxProb, _entropy)

    return [*contrast, *correlation, *energy, *homogeneity, *_maxProb, *_entropy]

In [48]:
def get_stats(b):
    return [np.min(b),
            np.percentile(b, .25), np.percentile(b, .5), np.percentile(b, .75),
            np.max(b), np.mean(b), np.std(b)]

In [49]:
extractors = ["contrast", "correlation", "energy", "homogeneity", "maxprob", "entropy"]
angles = ["np.pi/4", "0", "3*np.pi/2", "7*np.pi/4"]
statistics = ["min", "q25", "q50", "q75", "max", "mean", "std"]

# Laço de criação das Features

In [50]:
"""
Ground Truth (GT) is generated here.
"""

GT = np.zeros((n_images*512*512), dtype = "uint8")
for i in range(n_images):
    gt = imread(path_gts[i], as_grey=True)
    _max = np.max(gt)
    _max = _max if _max != 0 else 1
    #print(i, _max)
    gt = gt // _max
    GT[i*512*512: (i+1)*512*512] = gt.reshape(512*512)

In [51]:
"""
These variables are shared arrays to parallelize the feature creation
"""
features_names = names_VI = ["R", "G", "B"]

features_sharedArray = {f : Array("d", (n_images*512*512), lock=False) for f in features_names}

In [52]:
"""
This function is requied to use the shared variables as global variables in the shared enviroment.
"""

def _init(init_args):
    global tVI
    tVI = init_args[0]

In [53]:
"""
This is the function which generate all block instances (features of 16 x 16 block) of a given image
"""

def worker(i):
    
    VI = {f : np.frombuffer(tVI[f]).reshape(n_images*512*512) for f in features_names}
    
    print(i, end = ' ')
    img = img = imread(path_imgs[i], False)
    
    B, G, R = [np.float32(img[:, :, c]) for c in range(3)]
    r = R / (R + G + B)
    g = G / (R + G + B)
    b = B / (R + G + B)
    
    
    #VIs
    F = {} # Imagens Filtradas
    F["ExG"] = 2 * g - r - b
    #print("ExGR")
    F["ExGR"] = F["ExG"] - 1.4 * r - g
    #print("CIVE")
    F["CIVE"] = 0.441 * r - 0.881 * g + 0.385 * b + 18.78745
    #print("VEG")
    F["VEG"] = g / (2 + r ** 0.667 * b ** (1 - 0.667))
    #print("WI")
    F["WI"] = (g - b) / (r - g + 255) # MODIFICADO
    #print("NGRDI")
    F["NGRDI"] = (G/2 - R/2) / (G/2 + R/2) # Divide por dois pra evitar overflow            
    
    for f in names_VI:
        VI[f][i*512*512:(i+1)*512*512] = F[f].reshape(512*512)
        
    print("done")
    return i

In [54]:
"""
Define pool of processes
"""
pool = Pool(processes=4, initializer=_init, initargs=([sVI],))

2 done
4 done
0 done
6 done
5 done
3 done
1 done
7 done
8 done
12 done
14 done
10 done
9 done
13 done
11 done
15 done
18 done
20 done
16 done
19 done
17 done
21 done


In [55]:
"""
Execute pool of processes
"""
pool.map(worker, range(n_images))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [56]:
"""
Transform shared array structures into numpy variables
"""
VI = {f : np.frombuffer(sVI[f]) for f in names_VI}

In [57]:
VI

{'CIVE': array([18.62927055, 18.63313103, 18.63057709, ..., 18.74816322,
        18.74993324, 18.74921989]),
 'ExG': array([0.31099194, 0.30110496, 0.3054755 , ..., 0.04322204, 0.03913891,
        0.0410448 ]),
 'ExGR': array([-0.37747991, -0.37624311, -0.35561958, ..., -0.70333982,
        -0.7072407 , -0.70820892]),
 'NGRDI': array([0.4173913 , 0.42727274, 0.4589372 , ..., 0.09937888, 0.09597524,
        0.09411765]),
 'VEG': array([0.19585651, 0.19464915, 0.19610886, ..., 0.15053356, 0.14989524,
        0.15013592]),
 'WI': array([ 2.10484184e-04,  1.62661585e-04,  1.24448445e-04, ...,
        -7.70634942e-05, -8.44374445e-05, -7.31806649e-05])}

In [58]:
"""
Generate feature to identify image and block position of each block
"""
IMG = np.zeros(n_images*512*512, dtype="uint8")
for i in range(n_images):
    IMG[i*512*512:(i+1)*512*512] = i+0 # Base 2 começa no 0

In [59]:
data = pd.DataFrame(VI)

In [60]:
data["GT"] = GT
data["IMG"] = IMG

In [61]:
# data = pd.read_csv("dataset2.csv")

In [62]:
solo = np.zeros(len(data), int)
if base == 1:
    solo[data["IMG"] <= 18] = 0
    solo[data["IMG"] > 18] = 1
    data["solo"] = solo
    data["base"] = 0
if base == 2:
    solo = 2
    data["solo"] = solo
    data["base"] = 1

In [63]:
if base == 1:
    data.to_csv("PIXELdataset1.csv", index=False)
if base == 2:
    data.to_csv("PIXELdataset2.csv", index=False)

# Análises

# Análise das VIs (curva ROC)

In [ ]:
plt.figure(figsize=(12,9))
scores = pd.Series(None, index=data.columns)
for col in data.columns:
    score = roc_auc_score(GT, data[col])
    scores[col] = score
    print("%-30s AUC =" % col, score)
    fpr, tpr = roc_curve(GT, data[col])[:2]
    plt.plot(fpr,tpr, label = col)
plt.legend()

CIVE                           AUC = 0.18612729362208746
ExG                            AUC = 0.809777961025575
ExGR                           AUC = 0.836831508304779


In [37]:
scores.sort_values(ascending=False)

GT       1.000000
WI       0.958257
ExG      0.940711
VEG      0.934285
ExGR     0.900862
NGRDI    0.858256
base     0.500000
solo     0.344400
IMG      0.315439
CIVE     0.061969
dtype: float64

# Unificar bases

In [5]:
data1 = pd.read_csv("PIXELdataset1.csv")
data2 = pd.read_csv("PIXELdataset2.csv")

In [6]:
data2["IMG"] = data2["IMG"] + 40 # talvez não chegue a 40, mas vai ser o bastante

In [7]:
datafull = pd.concat([data1, data2])

In [8]:
datafull.shape

(15204352, 10)

In [9]:
datafull.to_csv("PIXELdatasetfull.csv", index=False)

In [40]:
data2["IMG"]

0        40
1        40
2        40
3        40
4        40
5        40
6        40
7        40
8        40
9        40
10       40
11       40
12       40
13       40
14       40
15       40
16       40
17       40
18       40
19       40
20       40
21       40
22       40
23       40
24       40
25       40
26       40
27       40
28       40
29       40
         ..
22498    61
22499    61
22500    61
22501    61
22502    61
22503    61
22504    61
22505    61
22506    61
22507    61
22508    61
22509    61
22510    61
22511    61
22512    61
22513    61
22514    61
22515    61
22516    61
22517    61
22518    61
22519    61
22520    61
22521    61
22522    61
22523    61
22524    61
22525    61
22526    61
22527    61
Name: IMG, dtype: int64